In [1]:
import pandas as pd
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_files = r"/content/drive/MyDrive/"

**Récupérer les fichiers d'outliers**

In [4]:
# récupérer les clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_outliers_minstp= glob.glob(f"{path_files}minstp/*")

In [5]:
list_outliers_lem= glob.glob(f"{path_files}lem/*")

In [6]:
list_outliers_std= glob.glob(f"{path_files}std/*")

In [7]:
import re

def extr_num(string):
    num = re.search("([0-9]+)", string)
    return int(num.group(1)) if num else -1

In [8]:
def make_liste(liste):
  liste_out=[]
  liste.sort(key=extr_num)
  for file in liste:
    liste_out.append(glob.glob(file+'/outliers*'))
  return liste_out

In [9]:
liste_files_minstp=make_liste(list_outliers_minstp)

In [10]:
liste_files_lem=make_liste(list_outliers_lem)

In [11]:
liste_files_std=make_liste(list_outliers_std)

**Fonction de lecture de fichiers**

In [12]:
def extr_num2(string):
    num = re.search("outliers([0-9]+)", string)
    return int(num.group(1)) if num else -1

In [13]:
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [14]:
def matrice(liste):
  cluster_km=[]
  for file in liste:
    file.sort(key=extr_num2)
    cluster_km.append(read_cluster(file))
  return cluster_km

In [15]:
matrice_outliers_minstp= matrice(liste_files_minstp)

In [16]:
matrice_outliers_lem= matrice(liste_files_lem)

In [17]:
matrice_outliers_std= matrice(liste_files_std)

**Fonction de detection de la haine les outliers de chaque cluster**

In [18]:
def search_hate_words(mlists, file):
    ret = {}
    with open(file, "r") as f:
        kws = f.read().splitlines()
    for i, mlist in enumerate(mlists):
        n_msg = 0
        for msg in mlist:
            for kw in kws:
                if kw in msg:
                    n_msg += 1
                    break
        if len(mlist)>0:
          perc = (n_msg / len(mlist))*100
          ret[i] = (n_msg, perc)
    return ret

In [19]:
def results(liste):
  res=[]
  for i in range(len(liste)):
    res.append(search_hate_words(liste[i],f"{path_files}keyword"))
  return res

In [20]:
# Enregistre un résumé des informations pour chaque cluster
def save_clusters_info(n,res,variante,wlists, method):
    f = open(path_files+"outliers_info"+variante+".txt", "a")
    f.write("methode {}\n".format(method))
    for i, key in enumerate(sorted(res)):
      if res[key][1]>10:
        f.write("cluster {} : {} messages soit {:.2f}%\n".format(key, res[key][0],res[key][1]))
        f.write("Les {} mots les plus fréquents : ".format(n))
        for tup in wlists[i][:n]:
          f.write(tup[0]+" ")
        f.write('\n')
    f.close()

In [21]:
liste_method=["km_25","km_50","km_100","som_100","som_225","som_400"]

In [22]:
def save_info_variante(n,res, variante,wlists,liste):
  for i in range(len(res)):
    save_clusters_info(n, res[i], variante,wlists[i] ,liste[i])

In [23]:
#Création du dictionnaire de mots avec leur occurence
def create_wdict(lines):
    dico = {}
    for msg in lines:
        for mot in msg.split():
            if mot in dico: 
                dico[mot] += 1
            else:
                dico[mot] = 1
    return dico

In [24]:
def create_sorted_wlist(lines):
    dico = create_wdict(lines)
    # Crée une liste de tuples associant mot et nombre d'occurences
    wlist = [(mot, dico[mot]) for mot in dico.keys()]
    # Renvoie la liste triée par nombre d'occurences décroissant
    return sorted(wlist, key = lambda mot: mot[1], reverse = True)

In [25]:
def liste_dic(matrice):
  liste_sorted=[[] for _ in range(len(matrice))]
  for i in range(len(matrice)):
    for n in range(len(matrice[i])):
      liste_sorted[i].append(create_sorted_wlist(matrice[i][n]))  
  return liste_sorted

**1- Cas d'application d'IF  sur les différents clusters de la version min stopword**

In [26]:
resultats_minstp=results(matrice_outliers_minstp)

In [27]:
srt_dics_minstp=liste_dic(matrice_outliers_minstp)

In [28]:
save_info_variante(20, resultats_minstp,"res_minstp",srt_dics_minstp, liste_method)

On remarque que deux clusters se détachent dans la version Km 100 avec des résultats avoisinant les 12% (cluster 85 et 94) , néanmoins une autre phase de clusterisation (SOM ou IF peut être appliqué dessus afin d'affiner la recherche

**2- Cas d'application d'IF  sur les différents clusters de la version avec lemmatisation**
___

In [29]:
resultats_lem=results(matrice_outliers_lem)

In [30]:
srt_dics_lem=liste_dic(matrice_outliers_lem)

In [31]:
save_info_variante(20, resultats_lem,"res_lem",srt_dics_lem, liste_method)

On remarque:

1.   le cluster 8 version som 100 avec 16% de mots haineux, (cluster homme femme noir)
2.   le cluster 30 version km 100 14%,(rasicte , haine , noir, arabe )
3.   le cluster 167 version som 400 11% (pute sale raciste)
4.   le cluster  156 version som 400 avec 37% (sur un cluster de 100 tweets) (noir , blanc , arabe )
5.   beaucoup de clusters vides sur le som 400 (à voir determiner le pourcentage de ces clusters)

d'autres cluster dans la version som 400 ont des proportions de mots haineux avoisinant les 10% à l'image du cluster 275, malheureusement il contient 2 message haineux ce qui veut dire que le cluster contient en tout et pour tout 20 messages au plus , les clusters étant éclatés , ils contiennent moins de message que les autres modèles , on se pose la question de la pertinence d'explorer ce genre de cluster , ceci étant dit , une exploration manuelle dudit cluster nous renseignera plus sur son contenu et sa pertinence .




****3- Cas d'application d'IF  sur les différents clusters de la version avec stemming****

---



In [32]:
resultats_std=results(matrice_outliers_std)

In [33]:
srt_dics_std=liste_dic(matrice_outliers_std)

In [34]:
save_info_variante(20, resultats_std,"res_std",srt_dics_std, liste_method)

1.   cluster 69 km100 15% environs (haine , noir rasicte haine)
2.   cluster 53 som100 20% environs (grossièrté (pute , sale et gros mots surtout)

som 225
3.   cluster 116: 11.5% (fake, certains tweets racistes (3 max ) et le cluster contient 26 tweets )

som400

4.   cluster 349: 23.40% (race arabe , blanc noir, sale pute)
5.   cluster 333: 26.92% (arabe blanc)
6.   cluster 89: 16.66% (3 messages) (à part 2 tweets contenant le mot chinois rien de particuluer sur ce cluster)


Nous remarquons que les meilleurs clusters sont donnés par la méthode SOM400 et KM100 et que les meilleures variantes sont la variante MIN et STD